# Описание проекта

* Необходимо подготовить прототип модели машинного обучения для «Цифры» - компании которая разрабатывает решения для эффективной работы промышленных предприятий.
* Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. В вашем распоряжении данные с параметрами добычи и очистки.
* Технология обогащения золота делится на 3 технологических процесса: флотация, первый этап очистки, второй этап очистки. На каждом из них концентрация золота растет, а примсей падает.
* Модель поможет оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками.

### Описание данных

### Признаки на входе в технологический процесс флотации
* rougher.input.feed_ag — концентрация серебра
* rougher.input.feed_pb — концентрация свинца
* rougher.input.feed_rate — скорость подачи
* rougher.input.feed_size — размер пульпы
* rougher.input.feed_sol — концентрация примесей
* rougher.input.feed_au — концентрация золота
* rougher.input.floatbank10_sulfate — реагент
* rougher.input.floatbank10_xanthate — реагент
* rougher.input.floatbank11_sulfate — реагент
* rougher.input.floatbank11_xanthate — реагент

### Целевые признаки
* rougher.output.recovery — эффективность обогащения чернового концентрата
* final.output.recovery — эффективность обогащения финального концентрата

## [Часть 1. Подготовка данных](#1-bullet)

## [Чачть 2. Анализ данных](#2-bullet)

## [Чачть 3. Модель](#3-bullet) 

## [Вывод ](#4-bullet) 

# 1. Подготовка данных <a id='1-bullet'></a>

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option('display.max_columns', None) 
import datetime
from scipy import stats as st
import plotly.express as px
import plotly.graph_objects as go
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score

In [2]:
# Функция проверки общего количества пропусков в % выражении
def number_of_passes(data):
    passes = pd.DataFrame(data.isnull().sum())
    passes['index'] = passes.index
    passes = passes.reset_index(drop =True)
    passes.columns = ['sum_NaN', 'specifications']
    passes = passes[['specifications', 'sum_NaN']]
    passes = passes.query('sum_NaN != 0').sort_values('specifications', ascending=False).reset_index(drop =True)
    passes['passes_NaN'] = round(passes['sum_NaN'] / data.shape[0] * 100, 1)
    return passes

In [3]:
# Ознакомимся с данными и количеством пропусков в них
df_train = pd.read_csv('C:/Users/vyugo/Documents/!Python/2. Проекты Я.Практикум/8. Модель по предсказанию коэффициента востановления золота из руды/gold_recovery_train.csv')
df_test = pd.read_csv('C:/Users/vyugo/Documents/!Python/2. Проекты Я.Практикум/8. Модель по предсказанию коэффициента востановления золота из руды/gold_recovery_test.csv')
df_full = pd.read_csv('C:/Users/vyugo/Documents/!Python/2. Проекты Я.Практикум/8. Модель по предсказанию коэффициента востановления золота из руды/gold_recovery_full.csv')

df_train_nan = number_of_passes(df_train)
df_test_nan = number_of_passes(df_test)
df_full_nan = number_of_passes(df_full)

#display(df_train_nan.head(50))
#display(df_test_nan.head())
#display(df_full_nan.head())

#df_test.head()
#df_train.head()
#df_train.info()

In [4]:
# Проверем, что эффективность обогащения рассчитана правильно найдем MAE на обучающей выборке
def recovery(df):
    C = df['rougher.output.concentrate_au']
    F = df['rougher.input.feed_au']
    T = df['rougher.output.tail_au']
    df['recovery.check'] = (C * (F - T)) / (F * (C - T)) * 100
    mae = sum(abs(df['rougher.output.recovery'] - df['recovery.check']).fillna(0)) / len(df)
    print('MAE:',mae)

In [5]:
# Проверим правильно ли рассчитано МАЕ в тестовой выборке
recovery(df_train)

MAE: 7.883623897364653e-15


In [6]:
# Найдем и проанализируем отсутствующие в тестовой выборке признаки
features = (pd.DataFrame(set(df_train.columns ^ df_test.columns))
                   .sort_values(by=0).reset_index(drop=True))
#display(features)
# Удалим из полученного списка целевые признаки ('rougher.output.recovery', 'final.output.recovery'), 
# для последующей фильтрации обучающих выборок
features_list = list(features.drop([4, 26])[0])

<ipython-input-6-4b65dcee1822>:2: FutureWarning: Index.__xor__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__xor__.  Use index.symmetric_difference(other) instead
  features = (pd.DataFrame(set(df_train.columns ^ df_test.columns))


In [7]:
# Преобразуем NaN в тренировочной выборке
df_train = df_train.fillna(method='ffill')

In [8]:
# Преобразуем date в индекс
df_train = df_train.set_index('date')
df_test = df_test.set_index('date')
df_full = df_full.set_index('date')

### Вывод
1. Исходная обучающая выборка состоит из 87 столбцов и 16860 строк.
2. Все столбцы имеют тип float, кроме столбца date - значения которого мы преобразуем в формат даты.
3. В виду того что данные содержат много пропусков, для расчета МАЕ (среднее абсолютное отклонение) мы удалили строки с пропусками. Полученное МАЕ стремиться к 0, что говорит о точности расчета эффективности обогащения.
4. Мы сравнили обучающую и тестовую выборки, было обнаружено 33 признака отсутствующих в тестовой выборке - по условиям задачи это целевые признаки трех циклов обогащения металлов + те что расчитываются позднее (calculetion).
5. Признаки содержащие output - это результаты каждлго из технологических циклов обогащения.
6. Наш целевой признак для создания модели rougher.output.recovery

# 2. Анализ данных <a id='2-bullet'></a>

In [9]:
# Функция по построению графиков
def f_boxplot_hist(df, column1, column2, column3, column4, name_histogram, name_box):
    #Построим диаграммы размаха
    fig1 = go.Figure()
    fig1.add_trace(go.Box(y=df[column1], name=column1))
    fig1.add_trace(go.Box(y=df[column2], name=column2))
    fig1.add_trace(go.Box(y=df[column3], name=column3))
    fig1.add_trace(go.Box(y=df[column4], name=column4))
    fig1.update_layout(
    title=name_box,
    title_x = 0.5,
    legend=dict(x=.1, xanchor="center", orientation="h"),
    barmode='overlay',
    margin=dict(l=0, r=0, t=30, b=0))
    fig1.show()
    # Построим гистораммы
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=df[column1], opacity=0.75, name=column1, histnorm='probability density'))
    fig.add_trace(go.Histogram(x=df[column2], opacity=0.75, name=column2, histnorm='probability density'))
    fig.add_trace(go.Histogram(x=df[column3], opacity=0.75, name=column3, histnorm='probability density'))
    fig.add_trace(go.Histogram(x=df[column4], opacity=0.75, name=column4, histnorm='probability density'))
    fig.update_layout(
    title=name_histogram,
    title_x = 0.5,
    xaxis_title="Концентрация",
    yaxis_title="Плотность",
    legend=dict(x=.1, xanchor="center", orientation="h"),
    barmode='overlay',
    margin=dict(l=0, r=0, t=30, b=0))
    fig.show()

In [10]:
#Построим графики концентраций Au на различных этапах очистки
# f_boxplot_hist(df_full,'rougher.input.feed_au', 
#                'rougher.output.concentrate_au', 
#                'primary_cleaner.output.concentrate_au',
#                'final.output.concentrate_au',
#                'Гистограмма концентраций Au', 'Диаграмма размаха концентрации Au')

### Вывод
1. На вышеприведенных графиках концентрации Au мы видим нормальное распределение концентрации на 3-х этапах обработки включая концентрацию в первоначальном сырье, что подтверждает адекватность наших данных. 
2. Каждый из этапов обработки золота демонстрирует рост концентрации, причем там где заканчиваются максимумы предидущего этапа начинаются минимумы следующего.
3. На гистограмме, если рассмотреть график каждого из этапов обработки отдельно (позволяет Plotly), хорошо заметны пики у 0 концентрации. Также выбросы продолжаются до первого квантиля каждого из этапов. Это нормальная ситуация в промышленности т.к. любой процесс химической/физической обработки имеет переходный характер т.е. на каждом этапе обработки определенного количества пульпы оборудование находиться в загруженной и незагруженной фазе - причем датчики непрерывно выдают показания. Когда физически в установке в начале каждого процесса обработки отсутствует пульпа - датчики будут выдавать 0 концентрации.
4. Отфильтруем данные по нижнему квантилю каждого из этапов.

In [11]:
# # Построим графики концентраций Pb на различных этапах очистки
# f_boxplot_hist(df_full,'rougher.input.feed_pb', 
#                'rougher.output.concentrate_pb', 
#                'primary_cleaner.output.concentrate_pb',
#                'final.output.concentrate_pb',
#                'Гистограмма концентраций Pb', 'Диаграмма размаха концентрации Pb')

### Вывод
1. На вышеприведенных графиках концентрации Pb мы видим нормальное распределение концентрации на 3-х этапах обработки включая концентрацию в первоначальном сырье, что подтверждает адекватность наших данных. 
2. Т.к. нашей задачей в этом технологическом процессе является повышение концентрации золота и понижение концентрации остальных веществ - это мы и видим на наших графиках. 
3. На каждом последующем этапе обработки среднее концентрации свинца падает вплоть до первого эиапа обработки и останавливается на 10 единицах. 
4. Далее подобным физикохимическим процессом понизить содержание свинца в концентрате золота невозможно - нужно искать другие методы.
5. По аналогичным причинам(как у золота) мы видим максимумы концентрации у 0 каждого из этапов обработки.

In [12]:
# # Построим графики концентраций Ag на различных этапах очистки
# f_boxplot_hist(df_full,'rougher.input.feed_ag', 
#                'rougher.output.concentrate_ag', 
#                'primary_cleaner.output.concentrate_ag',
#                'final.output.concentrate_ag',
#                'Гистограмма концентраций Ag', 'Диаграмма размаха концентрации Ag')

### Вывод
1. На вышеприведенных графиках концентрации Ag мы видим нормальное распределение концентрации на 3-х этапах обработки включая концентрацию в первоначальном сырье, что подтверждает адекватность наших данных. 
2. Т.к. нашей задачей в этом технологическом процессе является повышение концентрации золота и понижение концентрации остальных веществ - это мы и видим на наших графиках. Финальная средняя концентрация  Ag падает до 5 единиц.
3. По аналогичным причинам(как у золота и свинца) мы видим максимумы концентрации у 0 каждого из этапов обработки.

In [13]:
# # Построим графики концентраций Sol на различных этапах очистки
# f_boxplot_hist(df_full,'rougher.input.feed_sol', 
#                'rougher.output.concentrate_sol', 
#                'primary_cleaner.output.concentrate_sol',
#                'final.output.concentrate_sol',
#                'Гистограмма концентраций Sol', 'Диаграмма размаха концентрации Sol')

In [14]:
# Создадим функцию для построения диаграммы размаха размера гранул сырья на тренировочной, тестовой и общей выборках
def box(sign1, sign2, sign3, title, xaxis_title, yaxis_title):
    fig1 = go.Figure()
    fig1.add_trace(go.Box(y=sign1, name='Train'))
    fig1.add_trace(go.Box(y=sign2, name='Test'))
    fig1.add_trace(go.Box(y=sign3, name='Full'))
    fig1.update_layout(
    title=title,
    title_x = 0.5,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    #legend=dict(x=.2, xanchor="center", orientation="h"),
    barmode='overlay',
    margin=dict(l=0, r=0, t=30, b=0))
    fig1.show()

In [15]:
# Проверим распределение гранул сырья на всех наших выборках
# box(df_train['rougher.input.feed_size'], 
#     df_test['rougher.input.feed_size'],
#     df_full['rougher.input.feed_size'],
#     'График распределения', 'Распределение размера гранул сырья', 'Размер')

### Вывод
1. Распределение размера гранул сырья между тренировочной и тестовой выборками отличаются незначительно.
2. Распределение размера гранул сырья между полной выборкой и тренировочной почти отсутствуют.

In [16]:
# Функция подсчета суммарной концентрации на разных стадиях обработки
def total_concentration(processing_step): 
    drop_rate_size = df_full.drop(['rougher.input.feed_rate', 'rougher.input.feed_size'], axis=1)
    processing_step = list(filter(lambda x: x.startswith(processing_step), drop_rate_size.columns))
    return df_full[processing_step].sum(axis=1)   

In [17]:
# Суммарная концентрация подаваемого сырья
df_rougher_input = total_concentration('rougher.input.feed')
# Суммарная концентрация после этапа флотации
df_rougher_output = total_concentration('rougher.output.concentrate')
# Суммарная концентрация после первого этапа
df_primary_cleaner = total_concentration('primary_cleaner.output.concentrate')
# Суммарная концентрация после второго этапа
df_final_output = total_concentration('final.output.concentrate')

# Построим гисторамму всех этапов, Каждый этап можно рассмотреть отдельно переключаясь в Plotly
# fig = go.Figure()
# fig.add_trace(go.Histogram(x=df_rougher_input, opacity=0.75, name='Подаваемого сырья', histnorm='probability density'))
# fig.add_trace(go.Histogram(x=df_rougher_output, opacity=0.75, name='После этапа флотации', histnorm='probability density'))
# fig.add_trace(go.Histogram(x=df_primary_cleaner, opacity=0.75, name='После первого этапа', histnorm='probability density'))
# fig.add_trace(go.Histogram(x=df_final_output, opacity=0.75, name='После второго этапа', histnorm='probability density'))
# fig.update_layout(
# title="Концентрация всех веществ на разных этапах",
# title_x = 0.5,
# xaxis_title="Концентрация",
# yaxis_title="Плотность",
# legend=dict(x=.1, xanchor="center", orientation="h"),
# barmode='overlay',
# margin=dict(l=0, r=0, t=30, b=0))
# fig.show()

### Вывод
1. На гистограмме концентраций всех веществ на разных этапах мы видим нормальное распределение, что подтверждает адекватность наших данных. 
2. На гистограмме каждого из этапов обработки отдельно (позволяет Plotly), хорошо заметны пики у 0 концентрации. Это нормальная ситуация в промышленности т.к. любой процесс химической/физической обработки имеет переходный характер т.е. на каждом этапе обработки определенного количества пульпы оборудование находиться в загруженной и незагруженной фазе - причем датчики непрерывно выдают показания. Когда физически в установке в начале каждого процесса обработки отсутствует пульпа - датчики будут выдавать 0 концентрации.
4. Удалять выбросы имеет смысл только на обучающей выборке - чтобы наша модель в последствии не воспринимала их как достоверные значения.

In [18]:
# Удалим аномалии и выбросы из данных
df_train = df_train.loc[(1.711728 <= df_train['rougher.input.feed_au']) & (df_train['rougher.input.feed_au'] <= 14.09336) &
            (0.3485492 <= df_train['rougher.input.feed_pb']) & (df_train['rougher.input.feed_pb'] <= 6.52037) &
            (2.173518 <= df_train['rougher.input.feed_ag']) & (df_train['rougher.input.feed_ag'] <= 14.86965) &
            (21.80363 <= df_train['rougher.input.feed_sol']) & (df_train['rougher.input.feed_sol'] <= 50.20884)]
# Размер таблицы после удаления выбросов
df_train.shape

(14608, 87)

In [19]:
# Отфильтруем из выборок признаки которые не будут использоваться в обучении модели 
df_train = df_train.drop(features_list, axis = 1)

In [20]:
# Проверим результаты фильтрации
df_train[{'rougher.input.feed_au', 'rougher.input.feed_pb', 
                  'rougher.input.feed_ag', 'rougher.input.feed_sol'}].describe()

,rougher.input.feed_sol,rougher.input.feed_pb,rougher.input.feed_ag,rougher.input.feed_au
count,14608.000000,14608.000000,14608.000000,14608.000000
mean,36.485137,3.565754,8.698680,8.001774
std,4.340154,1.081851,1.925019,1.906028
min,21.803630,0.365561,2.598802,1.744413
25%,34.052179,2.802313,7.171959,6.669969
50%,36.988676,3.457345,8.243960,7.742544
75%,39.443914,4.286295,10.043812,9.229325
max,48.363177,6.520370,14.766530,13.923250


In [21]:
# Разделим выборку на признаки и целевые признаки
target_train_rougher = df_train['rougher.output.recovery'] 
target_train_final = df_train['final.output.recovery']
features_train = df_train.drop(['rougher.output.recovery', 'final.output.recovery'], axis=1)

In [22]:
# Подготовим тестовые признаки
features_test = df_test.dropna()
target_test_rougher = df_full['rougher.output.recovery'][features_test.index].fillna(method='ffill')
target_test_final = df_full['final.output.recovery'][features_test.index].fillna(method='ffill')

In [23]:
# Функция по удалению коррелированных признаков
def filter_df_corr(inp_data, corr_val):
       # Cоздадим матрицу корреляций
    if isinstance(inp_data, np.ndarray):
        inp_data = pd.DataFrame(data=inp_data)
        array_flag = True
    else:
        array_flag = False
    corr_matrix = inp_data.corr()

    # Найдем коррелированные столбцы
    drop_cols = []
    n_cols = len(corr_matrix.columns)

    for i in range(n_cols):
        for k in range(i+1, n_cols):
            val = corr_matrix.iloc[k, i]
            col = corr_matrix.columns[i]
            row = corr_matrix.index[k]
            if abs(val) >= corr_val:
                # Напечатаем коррелированные столбцы
                #print(col, "|", row, "|", round(val, 2))
                drop_cols.append(col)

    # Удалим коррелированные столбцы
    drop_cols = set(drop_cols)
    inp_data = inp_data.drop(columns=drop_cols)
    
    if array_flag:
        return inp_data.values
    else:
        return inp_data

In [24]:
# Удалим коррелированные признаки на тренировочной выборке
features_train = filter_df_corr(features_train, 0.85)

# Удалим коррелированные признаки на тестовой выборке
features_test = filter_df_corr(features_test, 0.85)

In [25]:
# Произведем масштабирование признаков тренировочной выборки
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)

# Произведем масштабирование признаков тренировочной выборки
scaler = StandardScaler()
scaler.fit(features_test)
features_test = scaler.transform(features_test)

# 3. Модель <a id='3-bullet'></a>

In [26]:
# Напишм функцию для вычисления sMAPE  «симметричное среднее абсолютное процентное отклонение», 
# Yi - целевой признак, Yii - предсказание 
def sMAPE(Yi, Yii): 
    return ((abs(Yi - Yii) / ((abs(Yi) + abs(Yii)) / 2)).mean()) * 100
# Создадим функцию оценки для GreedSearchCV
sMAPE_score_greed = make_scorer(sMAPE, greater_is_better = False)

# Создадим функцию оценки для модели
sMAPE_score = make_scorer(sMAPE)

### Дерево решений

In [27]:
# Запишем параметры модели
tree_params = {
    'max_depth': range(2,15),
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': range(2,5),
    'min_samples_split': range(2,5)}

In [28]:
# Найдем оптимальные гиперпараметры
tree = DecisionTreeRegressor(random_state=12345)
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, scoring=sMAPE_score_greed, verbose=True)
# Обучим модель
tree_grid.fit(features_train, target_train_rougher, target_train_final)
# печать оптимальных параметров
print(tree_grid.best_params_, tree_grid.best_score_) 

C:\Users\vyugo\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=date
2016-01-15 00:00:00    70.541216
2016-01-15 01:00:00    69.266198
2016-01-15 02:00:00    68.116445
2016-01-15 03:00:00    68.347543
2016-01-15 04:00:00    66.927016
                         ...    
2018-08-18 06:59:59    73.755150
2018-08-18 07:59:59    69.049291
2018-08-18 08:59:59    67.002189
2018-08-18 09:59:59    65.523246
2018-08-18 10:59:59    70.281454
Name: final.output.recovery, Length: 14608, dtype: float64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 351 candidates, totalling 1755 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 1755 out of 1755 | elapsed:   37.2s finished


{'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2} -11.432620558075374


In [29]:
# Мы получили следующие гиперпараметры {'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2} 
def tree(Ai, Bi, Ci, cv):
    tree = DecisionTreeRegressor(random_state=12345, max_depth=4, 
                                     max_features='sqrt', min_samples_leaf=2, min_samples_split=2)
    rougher_sMAPE = cross_val_score(tree, Ai, Bi, cv=cv, scoring=sMAPE_score)
    final_sMAPE = cross_val_score(tree, Ai, Ci, cv=cv, scoring=sMAPE_score) 
    return print('Итоговая sMAPE дерева решений:', round((0.25 * rougher_sMAPE + 0.75 * final_sMAPE).mean(), 2))

In [30]:
# Найдем sMAPE на тренировочной выборке
tree(features_train, target_train_rougher, target_train_final, 20)

Итоговая sMAPE дерева решений: 11.15


In [31]:
# Найдем sMAPE на тестовой выборке
tree(features_test, target_test_rougher, target_test_final, 20)

Итоговая sMAPE дерева решений: 9.99


### Случайный лес

In [32]:
forest_params = {
    'min_samples_split': range(2,4),
    'min_samples_leaf': range(2,5),
    'n_estimators': range(2, 20)}

In [33]:
# Найдем оптимальные гиперпараметры
forest = RandomForestRegressor(random_state=12345)
forest_class_weight = GridSearchCV(forest, forest_params, cv=5, n_jobs=-1, scoring=sMAPE_score_greed, verbose=True)
# Обучим модель
forest_class_weight.fit(features_train, target_train_rougher, target_train_final)
print(forest_class_weight.best_params_, forest_class_weight.best_score_) 

Fitting 5 folds for each of 108 candidates, totalling 540 fits


C:\Users\vyugo\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=date
2016-01-15 00:00:00    70.541216
2016-01-15 01:00:00    69.266198
2016-01-15 02:00:00    68.116445
2016-01-15 03:00:00    68.347543
2016-01-15 04:00:00    66.927016
                         ...    
2018-08-18 06:59:59    73.755150
2018-08-18 07:59:59    69.049291
2018-08-18 08:59:59    67.002189
2018-08-18 09:59:59    65.523246
2018-08-18 10:59:59    70.281454
Name: final.output.recovery, Length: 14608, dtype: float64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elap

{'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 14} -13.482878176617323


In [34]:
# Мы получили следующие гиперпараметры {'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 14}

# Напишем функцию по расчету sMAPE для модели случайного леса
# Ai - признаки; Bi - целевой признак rougher, Ci - целевой признак rougher final 
def forest(Ai, Bi, Ci, cv):
    model_forest = RandomForestRegressor(n_estimators = 14, min_samples_leaf=4, min_samples_split=2, random_state=12345)
    rougher_sMAPE = cross_val_score(model_forest, Ai, Bi, cv=cv, scoring=sMAPE_score)
    final_sMAPE = cross_val_score(model_forest, Ai, Ci, cv=cv, scoring=sMAPE_score) 
    return print('Итоговая sMAPE случайного леса:', round((0.25 * rougher_sMAPE + 0.75 * final_sMAPE).mean(), 2))

In [35]:
# Найдем итоговую sMAPE на тренировочной выборке для случайного леса
forest(features_train, target_train_rougher, target_train_final, 30)

Итоговая sMAPE случайного леса: 10.98


In [36]:
# Найдем итоговую sMAPE на тестовой выборке для случайного леса
forest(features_test, target_test_rougher, target_test_final, 30)

Итоговая sMAPE случайного леса: 9.76


### Алгоритм k-ближайших соседей

In [37]:
grid_params = {
    'n_neighbors': range(2,8),
    'weights': ['uniform'],
    'metric': ['minkowski']}

In [38]:
# Найдем оптимальные гиперпараметры
grid_knn = GridSearchCV(KNeighborsRegressor(), grid_params, cv=5, n_jobs=-1, scoring=sMAPE_score, verbose=1)
grid_knn.fit(features_train, target_train_rougher, target_train_final)
print(grid_knn.best_params_, grid_knn.best_estimator_, grid_knn.best_score_) 

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\vyugo\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=date
2016-01-15 00:00:00    70.541216
2016-01-15 01:00:00    69.266198
2016-01-15 02:00:00    68.116445
2016-01-15 03:00:00    68.347543
2016-01-15 04:00:00    66.927016
                         ...    
2018-08-18 06:59:59    73.755150
2018-08-18 07:59:59    69.049291
2018-08-18 08:59:59    67.002189
2018-08-18 09:59:59    65.523246
2018-08-18 10:59:59    70.281454
Name: final.output.recovery, Length: 14608, dtype: float64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   30.9s finished


{'metric': 'minkowski', 'n_neighbors': 2, 'weights': 'uniform'} KNeighborsRegressor(n_neighbors=2) 13.738379847868753


In [39]:
# Мы получили следующие гиперпараметры {'metric': 'minkowski', 'n_neighbors': 2, 'weights': 'uniform'}

# Напишем функцию по расчету sMAPE для модели KNN
# Ai - признаки; Bi - целевой признак rougher, Ci - целевой признак rougher final 
def KNN(Ai, Bi, Ci, cv):
    KNN_model = KNeighborsRegressor(n_neighbors = 2, weights='uniform', metric='minkowski')
    rougher_sMAPE = cross_val_score(KNN_model, Ai, Bi, cv=cv, scoring=sMAPE_score)
    final_sMAPE = cross_val_score(KNN_model, Ai, Ci, cv=cv, scoring=sMAPE_score) 
    return print('Итоговая sMAPE KNN:', round((0.25 * rougher_sMAPE + 0.75 * final_sMAPE).mean(), 2))

In [40]:
# Найдем итоговую sMAPE на тренировочной выборке для KNN
KNN(features_train, target_train_rougher, target_train_final, 30)

Итоговая sMAPE KNN: 12.74


In [41]:
# Найдем итоговую sMAPE на тестовой выборке для KNN
KNN(features_test, target_test_rougher, target_test_final, 30)

Итоговая sMAPE KNN: 10.67


# 3. Вывод<a id='4-bullet'></a>
* Дерево решений и Случайный лес дают почти одинаково хороший результат для предсказания итоговой концентрации золота на выходе технологического процесса, но алгоритм дерева решений выигрывает по скорости - рекомендуем его!